In [11]:
pip install folium

   ---------------------------------------- 0.0/110.5 kB ? eta -:--:--
   --- ------------------------------------ 10.2/110.5 kB ? eta -:--:--
   ------- ------------------------------- 20.5/110.5 kB 320.0 kB/s eta 0:00:01
   ---------------------------- ---------- 81.9/110.5 kB 762.6 kB/s eta 0:00:01
   -------------------------------------- 110.5/110.5 kB 911.6 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [31]:
import ee

# Initialize the Earth Engine API
ee.Initialize()

# Define the coordinates for the polygon
polygon_coordinates = [
    [17.5, 79.1],
    [17.6, 79.2],
    [17.7, 79.1],
    [17.6, 78.9]
]


# Create a polygon geometry from the coordinates
polygon = ee.Geometry.Polygon(polygon_coordinates)

# Function to calculate the cultivated area based on NDVI threshold
def is_cultivated(polygon):
    # Load the Sentinel-2 imagery collection
    image = ee.ImageCollection('COPERNICUS/S2') \
    .filterBounds(roi) \
    .filterDate('2021-06-01', '2021-09-30') \
    .sort('system:cloud_coverage') \
    .first()

    # Calculate NDVI (Normalized Difference Vegetation Index)
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')

    # Define the NDVI threshold for cultivated land (greater than 0.3 indicates vegetation)
    cultivated_mask = ndvi.gt(0.4)

    # Calculate the area of cultivated land (masked NDVI image)
    area = cultivated_mask.multiply(ee.Image.pixelArea()).reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=polygon,
        scale=10,
        maxPixels=1e8
    )

    # Extract the area value (in square meters)
    area_value = area.get('NDVI')

    # Check if the area of cultivated land is significant (greater than 0)
    if area_value.getInfo() > 0:
        return "Cultivated land"
    else:
        return "Not cultivated land"

# Check if the polygon is cultivated or not
result = is_cultivated(polygon)
print(result)


Not cultivated land


In [33]:
# Visualize the NDVI image for inspection
ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
ndvi_map = folium.Map(location=[17.6, 79.0], zoom_start=10)
ndvi_map.add_ee_layer(ndvi, {'min': -1, 'max': 1, 'palette': ['blue', 'white', 'green']}, 'NDVI')
ndvi_map

In [1]:
import ee

# Initialize the Earth Engine API
ee.Initialize()

# Define the coordinates for the polygon
polygon_coordinates = [
    [17.641192, 78.388979],
    [17.641197, 78.389090],
    [17.641089, 78.389093],
    [17.641092, 78.389010],
    [17.641092, 78.389010]
]

# Create a polygon geometry from the coordinates
polygon = ee.Geometry.Polygon(polygon_coordinates)

# Load the Sentinel-2 imagery collection
image = ee.ImageCollection('COPERNICUS/S2') \
    .filterBounds(polygon) \
    .filterDate('2021-06-01', '2021-09-30') \
    .sort('system:cloud_coverage') \
    .first()

# Check if the image is found
if image.getInfo() is None:
    print("No image found for the given region and date range.")
else:
    # Calculate NDVI (Normalized Difference Vegetation Index)
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')

    # Print the NDVI values for the polygon area
    ndvi_values = ndvi.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=polygon,
        scale=10,
        maxPixels=1e8
    )

    # Print the result
    print(ndvi_values.getInfo())


C:\Users\codhe\anaconda3\Lib\site-packages\ee\deprecation.py:207: DeprecationWarning: 

Attention required for COPERNICUS/S2! You are using a deprecated asset.
To ensure continued functionality, please update it.
Learn more: https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2

  warnings.warn(warning, category=DeprecationWarning)


{'NDVI': -0.05565465068625423}


In [55]:
pip install rasterio shapely numpy


   ---------------------------------------- 0.0/25.5 MB ? eta -:--:--
   ---------------------------------------- 0.1/25.5 MB 1.1 MB/s eta 0:00:24
    --------------------------------------- 0.6/25.5 MB 7.5 MB/s eta 0:00:04
   --- ------------------------------------ 2.2/25.5 MB 17.2 MB/s eta 0:00:02
   ---- ----------------------------------- 3.0/25.5 MB 17.3 MB/s eta 0:00:02
   ----- ---------------------------------- 3.6/25.5 MB 16.2 MB/s eta 0:00:02
   ------ --------------------------------- 4.0/25.5 MB 14.9 MB/s eta 0:00:02
   ------ --------------------------------- 4.1/25.5 MB 13.9 MB/s eta 0:00:02
   ------ --------------------------------- 4.4/25.5 MB 13.3 MB/s eta 0:00:02
   ------- -------------------------------- 4.7/25.5 MB 11.6 MB/s eta 0:00:02
   ------- -------------------------------- 5.0/25.5 MB 10.9 MB/s eta 0:00:02
   -------- ------------------------------- 5.2/25.5 MB 10.3 MB/s eta 0:00:02
   -------- ------------------------------- 5.4/25.5 MB 9.8 MB/s eta 0:00

In [1]:
import ee

# Initialize the Earth Engine library
ee.Initialize()

# Define the polygon coordinates
polygon_coordinates = [
    # [78.388979, 17.641192],
    # [78.389090, 17.641197],
    # [78.389093, 17.641089],
    # [78.389010, 17.641092]
    [17.636414, 78.392036],
    [17.636445, 78.393055],
    [17.635659, 78.393055],
    [17.635515, 78.391950],
    [17.635515, 78.391950]
]

# Create a geometry for the polygon
polygon = ee.Geometry.Polygon([polygon_coordinates])

# Load Sentinel-2 surface reflectance data
sentinel2 = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") \
    .filterBounds(polygon) \
    .filterDate("2023-01-01", "2023-12-31") \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)) \
    .median()  # Combine multiple images

# Select Red (B4) and NIR (B8) bands
red = sentinel2.select('B4')
nir = sentinel2.select('B8')

# Calculate NDVI
ndvi = nir.subtract(red).divide(nir.add(red)).rename('NDVI')

# Reduce the NDVI image over the polygon to get the mean value
ndvi_mean = ndvi.reduceRegion(
    reducer=ee.Reducer.mean(),
    geometry=polygon,
    scale=10
).get('NDVI')

# Fetch and print the result
print("Calculating NDVI value...")
ndvi_value = ndvi_mean.getInfo()
print(f"NDVI Value for the polygon: {ndvi_value}")

Calculating NDVI value...
NDVI Value for the polygon: 0.08911735179041964
